User provides amazon url for an item and a desired price point. The program scrapes the price and if the price is ever at or below the buy point, emails the user.

# Scrape price from Amazon

In [1]:
import requests as re
from bs4 import BeautifulSoup
import smtplib
from email.message import EmailMessage
import os
from dotenv import load_dotenv

load_dotenv() #get environment variables from .env

TARGET_PRICE = 100
amazon_url = 'https://appbrewery.github.io/instant_pot/'

HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "en-US,en;q=0.9",  
    "Priority": "u=0, i", 
    "Sec-Ch-Ua": "\"Chromium\";v=\"128\", \"Not;A=Brand\";v=\"24\", \"Google Chrome\";v=\"128\"", 
    "Sec-Ch-Ua-Mobile": "?0", 
    "Sec-Ch-Ua-Platform": "\"Windows\"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Chrome/128.0.0.0",
  }

In [2]:
#code to retrieve html of product page, convert to soup object, extract price and product title

response = re.get(amazon_url,headers=HEADERS)
soup = BeautifulSoup(response.text, 'html.parser')

dollars = soup.select_one('.a-price-whole').text
cents = soup.select_one('.a-price-fraction').text

product_title = soup.select_one('#productTitle').text.split("\r")[0].strip()
print(f'${dollars}{cents}')
print(product_title)

$99.99
Instant Pot Duo Plus 9-in-1


In [3]:
#function to send email notifying user of sale
def send_email(price,product):
    my_email = os.environ["EMAIL_ADDRESS"]
    app_pw = os.environ["EMAIL_PASSWORD"]
    msg = EmailMessage()
    msg['Subject'] = 'Amazon Sale Notice'
    msg['From'] = my_email
    msg['To'] = my_email
    msg.set_content(f'Amazon item: {product} \nPrice: ${price}\nTarget Price: ${TARGET_PRICE}')

    with smtplib.SMTP(os.environ['SMTP_ADDRESS'],port=587) as connection:
        connection.starttls()
        connection.login(user = my_email, password = app_pw)
        connection.send_message(msg)

In [5]:
if float(f'{dollars}{cents}') < TARGET_PRICE:
    send_email(f'{dollars}{cents}',product_title)